### Load and Clean Data


In [10]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [14]:
df = pd.read_csv('Music.csv', header=0, low_memory=False)
# Drop irrelevant columns
df = df.drop(['id', 'track_id'], axis=1)
df = df.dropna()
#print(df.columns)

# Normalize features
scaler = MinMaxScaler()
features = ['valence', 'energy', 'tempo']
df[features] = scaler.fit_transform(df[features])
print(df.head())

                  artists                                         album_name  \
0             Gen Hoshino                                             Comedy   
1            Ben Woodward                                   Ghost (Acoustic)   
2  Ingrid Michaelson;ZAYN                                     To Begin Again   
3            Kina Grannis  Crazy Rich Asians (Original Motion Picture Sou...   
4        Chord Overstreet                                            Hold On   

                   track_name  popularity  duration_ms  explicit  \
0                      Comedy          73       230666     False   
1            Ghost - Acoustic          55       149610     False   
2              To Begin Again          57       210826     False   
3  Can't Help Falling In Love          71       201933     False   
4                     Hold On          82       198853     False   

   danceability  energy  key  loudness  mode  speechiness  acousticness  \
0         0.676  0.4610    1    -6.